In [ ]:
# MIE1621 Fall 2017
# Project 

import numpy as np
import matplotlib.pyplot as plt


def pprint(obj):
    '''
    For debugging, print statements with numpy variable names and shape
    '''
    def namestr(obj):
        namespace = globals()
        return [name for name in namespace if namespace[name] is obj]
    # Assumes obj is a numpy array, matrix
    try:
        print(namestr(obj), obj.shape)
    except:
        try:
            print(namestr(obj), ",", len(obj))
        except:
            print(namestr(obj))
    print(obj)


covarianceMatrix = np.array([[0.02778, 0.00387, 0.00021],
                             [0.00387, 0.01112, -0.00020],
                             [0.00021, -0.00020, 0.00115]])

expectedReturn = np.array([0.1073, 0.0737, 0.0627])

stepSize = 1.0

delta = 4.0

# Randomly initialize to anything
initialX = np.array([0.25, 0.25, 0.5])
initialPi = 1.2

def checkShape(x, u, sigma, delta):
    if delta <= 0.0:
        raise ValueError("Risk aversion, delta needs to be > 0")
    if delta <= 3.5 or delta >= 4.5:
        raise ValueError("Risk aversion, delta needs to be between (3.5, 4.5)")
    if sigma.shape[0] != sigma.shape[1]:
        raise ValueError("Covariance matrix must be square")
    if sigma.shape[0] != x.shape[0]:
        raise ValueError("Dimensions dont match between sigma and x")
    if u.shape[0] != x.shape[0]:
        raise ValueError("Dimensions dont match between u and x")
    '''
    Not always met, only met after training is over
    if np.sum(x) != 1.0:
        raise ValueError("X must sum to 1.0")
    '''

def computeF(x, u, sigma, delta):
    '''
    x is the proportion, the parameters we are changing to maximize f
    u is the expected return
    sigma is the covariance matrix
    delta is the parameter that controls risk aversion, delta > 0
    '''
    checkShape(x, u, sigma, delta)
    
    firstTerm = 0.0
    for (ui, xi) in zip(u, x):
        firstTerm += ui * xi
    secondTerm = 0.0
    for i in range(x.shape[0]):
        for j in range(x.shape[0]):
            secondTerm += sigma[i][j] * x[i] * x[j]
    secondTerm *= (float(delta)/float(2.0))
            
    f = firstTerm - secondTerm
    return f

def computeGrad(x, pi, u, sigma, delta):
    checkShape(x, u, sigma, delta)
    grad = np.zeros((x.shape[0] + 1, 1))
    dLdx = np.zeros(x.shape)
    for i in range(x.shape[0]): 
        secondIndex = (i + 1) % x.shape[0]
        thirdIndex = (i + 2) % x.shape[0]
        
        dLdx[i] = u[i] + pi - ((delta)*((float(sigma[i][i]*x[i])/2.0) \
                                        + sigma[i][secondIndex]*x[secondIndex]) \
                               + sigma[i][thirdIndex] * x[thirdIndex])
        grad[i] = dLdx[i]
        
    dLdPi = np.sum(x) - 1.0
    grad[x.shape[0]] = dLdPi
    
    return grad; 

def computeHessian(x, pi, u, sigma, delta):
    checkShape(x, u, sigma, delta)
    dim = x.shape[0] + 1
    hessian = np.zeros((dim, dim))
    for i in range(x.shape[0]): 
        for j in range(x.shape[0]): 
            hessian[i][j] = (-1.0) * delta * sigma[i][j]
            if i == j:
                hessian[i][j] /= 2.0
    for i in range(x.shape[0]): 
        hessian[x.shape[0]][i] = 1.0
        hessian[i][x.shape[0]] = 1.0
    return hessian

x = initialX.copy()
pi = initialPi 
f = computeF(x, expectedReturn, covarianceMatrix, delta)

pprint(covarianceMatrix)
pprint(expectedReturn)
pprint(x)
pprint(f)

grad = computeGrad(x, pi, expectedReturn, covarianceMatrix, delta)
pprint(grad)

hessian = computeHessian(x, pi, expectedReturn, covarianceMatrix, delta)
pprint(hessian)

maxIteration = 1000

In [ ]:
# Newton Method
currX = x.copy()
currPi = pi
for iteration in range(maxIteration):
    pprint(iteration)
    
    x = currX.copy()
    pi = currPi
    
    f = computeF(x, expectedReturn, covarianceMatrix, delta)
    constraint = np.sum(x)
    pprint(f)
    pprint(constraint)
    pprint(x)
    pprint(pi)
    
    grad = computeGrad(x, pi, expectedReturn, covarianceMatrix, delta)
    hessian = computeHessian(x, pi, expectedReturn, covarianceMatrix, delta)
    
    direction = np.dot(np.linalg.inv(hessian), grad)
    pprint(direction)
    
    # Stopping condition
    if np.linalg.norm(direction) < np.power(0.1, 10):
        break
    xAdd = np.reshape(direction[:-1].copy(), currX.shape)
    
    # You always deduct in Newton Method!
    # As newton method accounts for correct direction regardless of max or min.
    # Newton's method always searches for critical points. 
    currX -= stepSize * xAdd
    currPi -= stepSize * direction[-1]

In [ ]:
# Steepest Descent Method
# Note: NOT really steepest as the question asks to use stepsize of 1.0, so don't calculate stepsize. 
# Just do the normal gradient descent in the Machine Learning literature

currX = x.copy()
currPi = pi
for iteration in range(maxIteration):
    pprint(iteration)
    
    x = currX.copy()
    pi = currPi
    
    f = computeF(x, expectedReturn, covarianceMatrix, delta)
    constraint = np.sum(x)
    pprint(f)
    pprint(constraint)
    pprint(x)
    pprint(pi)
    
    grad = computeGrad(x, pi, expectedReturn, covarianceMatrix, delta)
    # No hessian computation needed for gradient descent
    
    direction = grad
    pprint(direction)
    
    # Stopping condition
    if np.linalg.norm(direction) < np.power(0.1, 10):
        break
    xAdd = np.reshape(direction[:-1].copy(), currX.shape)
    
    # Gradient Descent diverges with constant step size
    # stepSize = 0.00001 , can only converge with small step size but not OPTIMAL
    # Need to add cause maximizing the function, NOT minimizing
    currX += stepSize * xAdd
    currPi += stepSize * direction[-1]


In [ ]:
# Quasi Newton Methods 
# BFGS

H = np.eye(x.shape[0] + 1)

def BFGSUpdate(H, y, s): 
    Hnext = H + (1.0/(np.dot(np.transpose(y), s))) * np.dot(y, np.transpose(y)) - (1.0/(np.dot(np.dot(np.transpose(s), H), s))) * np.dot(np.dot(np.dot(H, s), np.transpose(s)), H)
    return Hnext

pprint(x)
pprint(pi)
xnew = np.zeros((x.shape[0]+1,1))
for i in range(x.shape[0]):
    xnew[i] = x[i]
xnew[x.shape[0]] = pi

pprint(xnew)

for iteration in range(maxIteration):
    xold = xnew
    gradFk = computeGrad(xold[:-1], xold[-1], expectedReturn, covarianceMatrix, delta)
    direction = np.dot(np.linalg.inv(H), gradFk)
    # Stopping condition
    if np.linalg.norm(direction) < np.power(0.1, 10):
        break
    #----------------------------------------------------------
    xnew = xold - stepSize * direction
    gradFk1 = computeGrad(xnew[:-1], xnew[-1], expectedReturn, covarianceMatrix, delta)
    s = xnew - xold
    y = gradFk1 - gradFk
    # BFGS Update
    Hnext = BFGSUpdate(H,y,s)
    H = Hnext
    
    f = computeF(xnew[:-1], expectedReturn, covarianceMatrix, delta)
    pprint(f)
    pprint(iteration)
    pprint(xold)
    pprint(H)
    pprint(gradFk)
    pprint(direction)
    pprint(xnew)
    pprint(gradFk1)
    pprint(y)
    pprint(s)

In [ ]:
# Newton Method with Backtracking

In [ ]:
# Steepest Descent Method with Backtracking

In [ ]:
# Quasi Newton Methods with Backtracking
# BFGS